# 《数据采集与清洗》
## 第一次作业内容： 网页抓取
### 具体目标：
+ 用``urllib.requests``库发起一次``get``请求，输出``response``文本信息；
+ 用``Requests``库伪装成火狐浏览器发起一次``post``请求，输出``response``文本信息；
+ 用``Requests``库中的``session``对象发出``get``请求，设置``cookies``并获取，输出获取的``cookies``内容；
+ 实现抓取网页的不去重深度遍历算法，自选合适的种子网站和相关参数，输出结果；
+ 编写抓取网页的广度遍历算法（含去重和不去重），自选合适的种子网站和相关参数，输出结果；
+ (选)将抓取网页的去重深度遍历算法封装成对象（类），并测试。

### 注：
+ 代码要有注释，结果要有分析；
+ 本次作业提交截至时间：2020年3月10日(星期二)；
+ 文件命名规则: 班级号+学号+姓名+作业序号，示例：``1_20188989899_张三_1``；
+ 提交方式：1班发送至邮箱：632994085@qq.com；2班发送至邮箱：786888939@qq.com.

### 问题1：用``urllib.requests``库发起一次``get``请求，输出``response``文本信息。

In [1]:
# 这里编写代码

In [1]:
import urllib.request
import urllib.parse
data = urllib.parse.urlencode({'word': 'hello'}).encode()#get请求的参数
response = urllib.request.urlopen('http://httpbin.org/post',data=data)#发送请求
print(response.read().decode())#将得到的响应解码

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "word": "hello"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "10", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7", 
    "X-Amzn-Trace-Id": "Root=1-5e66739c-8442c5f8bb1f07316c887110"
  }, 
  "json": null, 
  "origin": "110.53.234.42", 
  "url": "http://httpbin.org/post"
}



+ 这里对结果进行分析。

**"headers"代表请求头**

Accept-Encoding：客户端支持的压缩方式

Content-Length：消息体的长度为10

Content-Type：客户端告诉服务器实际发送的数据类型

Host：要请求的服务器域名，与url一致

User-Agent客户端标识

### 用``Requests``库伪装成火狐浏览器发起一次``post``请求，输出``response``文本信息。

In [2]:
# 这里编写代码

In [5]:
import requests,re
url="http://httpbin.org/post"
headers={
    "User-Agent":'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',
    "host":'httpbin.org'
         }#伪装一个火狐浏览器
dict={"name":"Hackdata"}
response=requests.post(url,params=dict,headers=headers)#发起post请求
print(response.text)

{
  "args": {
    "name": "Hackdata"
  }, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "0", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)", 
    "X-Amzn-Trace-Id": "Root=1-5e667e45-dc6bece86fb204b8bc0fcc14"
  }, 
  "json": null, 
  "origin": "110.53.234.42", 
  "url": "http://httpbin.org/post?name=Hackdata"
}



+ 这里对结果进行分析。

**"headers"后为请求头**

Accept:* / * 表示客户端支持任意的MIME类型

Accept-Encoding: 客户端支持gzip, deflate这两种压缩方式

Host:要请求的服务器域名"httpbin.org"

User-Agent:客户端标识

X-Amzn-Trace-Id":可记录此唯一标识符，以解决与负载均衡器相关的问题

### 用``Requests``库中的``session``对象发出``get``请求，设置``cookies``并获取，输出获取的``cookies``内容。 

In [3]:
# 这里编写代码

In [10]:
s = requests.Session() # 创建一个session对象 
s.get('http://httpbin.org/cookies/set/sessioncookie/123456789')
                                        # 用session对象发出get请求，设置cookies 
r = s.get("http://httpbin.org/cookies") # 用session对象发出另外一个get请求，获取cookies 
print(r.text)# 显示结果 
print(r.request.headers)#查看发送请求的请求头

{
  "cookies": {
    "sessioncookie": "123456789"
  }
}

{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': 'sessioncookie=123456789'}


+ 这里对结果进行分析。

requests库的session对象能够帮我们跨请求保持某些参数，也会在同一个session实例发出的所有请求之间保持cookies。

Cookie:在客户端访问Web服务器时，服务器在客户端硬盘上存放的信息

当客户端首次请求访问服务器时，服务器先在客户端存放包含该客户的相关信息的Cookie，以后客户端每次请求访问服务器时，都会在HTTP请求数据中包含Cookie，服务器解析HTTP请求中的Cookie，就能由此获得关于客户的相关信息

### 实现抓取网页的不去重深度遍历算法，自选合适的种子网站和相关参数，输出结果。


In [9]:
# 这里编写代码

In [66]:
import urllib.request as ur
import re
r = ur.urlopen("https://baike.baidu.com/item/"
               "%E7%B4%AB%E8%8B%8F%E5%B1%9E") # 对百度百科关键词条为紫苏属的URL的进行访问
content = r.read().decode('utf-8')
href = re.compile(r'href=[\'"]?(/item[^\'" >]+)') # 利用正则表达式将网页中所需的链接表达出来
new_urls=href.findall(content) # 使用findall方法将所有链接信息抽取出来
#print(new_urls) 打印该网页中的所有链接

#实现深度优先爬取
count = 0
r = re.compile(r'href=[\'"]?(/item[^\'" >]+)') # 抽取所需链接信息的正则语言规则
seed = "/item/%E7%B4%AB%E8%8B%8F%E5%B1%9E" # 这是紫苏属词条
stack = [seed] # 设置种子链接的栈（使用列表模拟栈）
storage = {}
while count < 3:
    try:
        url = stack.pop(-1) # 取出栈的最后一条URL
        html = ur.urlopen("https://baike.baidu.com"+url).read().decode('utf-8') # 对URL进行拼接
        new_urls = r.findall(html) # 提取当前网页下的所有链接URL信息
        print(new_urls)
        stack.extend(new_urls) # 将新提取的链接信息入队列
        storage[url] = len(new_urls)
        count += 1
    except Exception as e:
        print(url)
        print(e)
        
        
#加层数控制的
count = 0 # 层数
floors = 1 # 限制爬取的层数
lastStep = []
r = re.compile(r'href=[\'"]?(/item[^\'" >]+)')
seed = '/item/%E7%B4%AB%E8%8B%8F%E5%B1%9E' # 这是紫苏属词条
queue = [[seed]]
for i in range(floors): # 限制爬取范围在设定的层数范围内
    queue.append([])
storage = {}
used = set() # 设置集合存放爬取过的url
while len(queue[0])>0 or count!=0: # 种子队列不为空或者层数不为零
    try:
        url = queue[count].pop(-1)
        print(url+" "+str(count)) # 打印当前链接和层数
        html = ur.urlopen('https://baike.baidu.com'+url).read().decode('utf-8')
        storage[url]=html
        used.add(url) # 将爬取过的URL放入集合中
        new_urls = r.findall(html)
        if count < floors:
            for new_url in set(new_urls):
                if new_url not in used and new_url not in queue:#判断新链接网址中的包含的链接是否为重复的
                    queue[count+1].append(new_url) # 将爬取的URL存入到队列中相应层数的列表
            count+=1
        else:
            if len(queue[count])==0:
                count -= 1
    except Exception as e:
        print(url)
        print(e)

['/item/秒懂星课堂', '/item/秒懂大师说', '/item/秒懂看瓦特', '/item/秒懂五千年', '/item/秒懂全视界', '/item/%E7%B4%AB%E8%8B%8F/511', '/item/%E8%8B%8F%E5%AD%90%E6%B2%B9/8682670', '/item/%E8%8D%89%E6%9C%AC', '/item/%E6%80%BB%E7%8A%B6%E8%8A%B1%E5%BA%8F', '/item/%E8%8B%9E%E7%89%87', '/item/%E6%8A%AB%E9%92%88%E5%BD%A2', '/item/%E5%9D%9A%E6%9E%9C/80641', '/item/%E7%B4%AB%E8%8B%8F/511', '/item/%E9%87%8E%E7%94%9F%E7%B4%AB%E8%8B%8F', '/item/%E8%80%B3%E9%BD%BF%E5%8F%98%E7%A7%8D', '/item/%E5%9B%9E%E5%9B%9E%E8%8B%8F', '/item/%E7%99%BE%E5%BA%A6%E7%99%BE%E7%A7%91%EF%BC%9A%E6%9C%AC%E4%BA%BA%E8%AF%8D%E6%9D%A1%E7%BC%96%E8%BE%91%E6%9C%8D%E5%8A%A1/22442459?bk_fr=pcFooter']
['/item/秒懂星课堂', '/item/秒懂大师说', '/item/秒懂看瓦特', '/item/秒懂五千年', '/item/秒懂全视界', '/item/%E7%99%BE%E5%BA%A6%E7%99%BE%E7%A7%91/85895', '/item/%E5%8F%82%E8%80%83%E8%B5%84%E6%96%99/5718004', '/item/%E8%BA%AB%E4%BB%BD%E8%AF%81/113951', '/item/%E6%88%B7%E5%8F%A3%E6%9C%AC/5104621', '/item/%E4%B9%89%E9%A1%B9/6176882', '/item/%E5%8F%82%E8%80%83%E8%B5%84%E6%96%99/5718004', '

+ 这里对结果进行分析。

从结果看出，由于网页的最后一个链接都是相同的，所以在第一次爬取后再爬取的所有信息都将是同一个链接，即爬取的是重复信息，而且爬虫将陷入无限循环。

### 编写抓取网页的广度遍历算法（含去重和不去重），自选合适的种子网站和相关参数，输出结果。 

In [5]:
# 这里编写代码

In [63]:
#或者使用广度优先爬取
count = 0
r = re.compile(r'href=[\'"]?(/item[^\'" >]+)') # 抽取所需链接信息的正则语言规则
seed = "/item/%E7%B4%AB%E8%8B%8F%E5%B1%9E" # 这是紫苏属词条
queue = [seed] # 设置种子链接的队列（使用列表模拟队列）
storage = {}
while count < 2:
    try:
        url = queue.pop(0) # 取出队列最后一条URL
        html = ur.urlopen("https://baike.baidu.com"+url).read().decode('utf-8') # 对URL进行拼接
        new_urls = r.findall(html) # 提取当前网页下的所有链接URL信息
        print(new_urls)
        queue.extend(new_urls) # 将新提取的链接信息入队列
        storage[url] = len(new_urls)
        count += 1
    except Exception as e:
        print(url)
        print(e)

        
#加层数控制的
count = 0 # 层数
floors = 1 # 限制爬取的层数
lastStep = []
r = re.compile(r'href=[\'"]?(/item[^\'" >]+)')
seed = '/item/%E7%B4%AB%E8%8B%8F%E5%B1%9E' # 这是紫苏属词条
queue = [[seed]]
for i in range(floors): # 限制爬取范围在设定的层数范围内
    queue.append([])
storage = {}
used = set() # 设置集合存放爬取过的url
while len(queue[0])>0 or count!=0: # 种子队列不为空或者层数不为零
    try:
        url = queue[count].pop(-1)
        print(url+" "+str(count)) # 打印当前链接和层数
        html = ur.urlopen('https://baike.baidu.com'+url).read().decode('utf-8')
        storage[url]=html
        used.add(url) # 将爬取过的URL放入集合中
        new_urls = r.findall(html)
        if count < floors:
            for new_url in set(new_urls):
                if new_url not in used and new_url not in queue:#判断新链接网址中的包含的链接是否为重复的
                    queue[count+1].append(new_url) # 将爬取的URL存入到队列中相应层数的列表
            count+=1
        else:
            if len(queue[count])==0:
                count -= 1
    except Exception as e:
        print(url)
        print(e)

['/item/秒懂星课堂', '/item/秒懂大师说', '/item/秒懂看瓦特', '/item/秒懂五千年', '/item/秒懂全视界', '/item/%E7%B4%AB%E8%8B%8F/511', '/item/%E8%8B%8F%E5%AD%90%E6%B2%B9/8682670', '/item/%E8%8D%89%E6%9C%AC', '/item/%E6%80%BB%E7%8A%B6%E8%8A%B1%E5%BA%8F', '/item/%E8%8B%9E%E7%89%87', '/item/%E6%8A%AB%E9%92%88%E5%BD%A2', '/item/%E5%9D%9A%E6%9E%9C/80641', '/item/%E7%B4%AB%E8%8B%8F/511', '/item/%E9%87%8E%E7%94%9F%E7%B4%AB%E8%8B%8F', '/item/%E8%80%B3%E9%BD%BF%E5%8F%98%E7%A7%8D', '/item/%E5%9B%9E%E5%9B%9E%E8%8B%8F', '/item/%E7%99%BE%E5%BA%A6%E7%99%BE%E7%A7%91%EF%BC%9A%E6%9C%AC%E4%BA%BA%E8%AF%8D%E6%9D%A1%E7%BC%96%E8%BE%91%E6%9C%8D%E5%8A%A1/22442459?bk_fr=pcFooter']
/item/秒懂星课堂
'ascii' codec can't encode characters in position 10-14: ordinal not in range(128)
/item/秒懂大师说
'ascii' codec can't encode characters in position 10-14: ordinal not in range(128)
/item/秒懂看瓦特
'ascii' codec can't encode characters in position 10-14: ordinal not in range(128)
/item/秒懂五千年
'ascii' codec can't encode characters in position 10-14: ordinal 

+ 这里对结果进行分析。

当网页链接中存在相同链接时，有必要对URL进行去重处理。即我们可以将爬取过的URL存放在一个元素集合中，在进行新的爬取之前将目标URL与爬取过的集合进行对比，只爬取元素集合中没有的URL，就可以完成去重处理了。并且设置限制爬取链接的层数。

### (选)将抓取网页的去重深度遍历算法封装成对象（类），并测试。 

In [6]:
# 这里编写代码

In [7]:
# 测试

这里对结果进行分析。